In [2]:
import sys
sys.path.append("..")
from utils.SolutionChecker import Case, check_solution

# Bracket Expansion

### Problem Description

Implement a similar behavior to bash's brace expansion behavior as a runnable program.

For a valid input, print the output. For an invalid input, print nothing and exit.

### Considerations

  - Any input without properly matching braces is invalid
  - Commas should only appear within braces. 
  - Restrict the input character set to \[a-zA-Z\{\},\] 
  - Braces should not be empty, and there should be no "empty" options within braces i.e. {A,}

### Examples of Valid Input
  - {A,B,C} -> A B C
  - AB{C,D} -> ABC ABD
  - {A,B}{C,D} -> AC AD BC BD
  - {A,B{C,D}} -> A BC BD
  - {{A},{B}} -> A B
  - {ABC} -> ABC
  - ABC -> ABC

###  Examples of invalid input
  - }ABC
  - {ABC
  - }{
  - {}
  - A,B,C
  - A B C
  - {A{B,C}
  - {A,}


In [15]:
cases = [Case('{A,B,C}','A B C'), 
         Case('AB{C,D}', 'ABC ABD'),
         Case('{C,D}AB', 'CAB DAB'),
         Case('{A,B}{C,D}', 'AC AD BC BD'),
         Case('{A,B{C,D}}', 'A BC BD'), 
         Case('{{A},{B}}','A B'),
         Case('{ABC}','ABC'),
         Case('{ABC',None),
         Case('{A,B,C}{A,B{{A}}}','AA ABA BA BBA CA CBA'), 
         Case('D{A,B}{A,B{{C}}}','DAA DABC DBA DBBC'), 
         Case('{A,B}{{C,D}EF}','ACEF ADEF BCEF BDEF'), 
         Case('AB{C,D}', 'ABC ABD'),
         Case('{A}{B}{C,D}', 'ABC ABD'),
         Case('A,B{C,D}', None), 
         Case('{{A},{B}}','A B'),
         Case('{ABC}','ABC'),
         Case('}ABC',None),
         Case('}{',None),
         Case('{}',None),
         Case('A,B,C',None),
         Case('{A{B,C}',None),
         Case('{A,',None),
         Case('A{?}',None),
         Case('',None),
        ]

In [16]:
def merge_values(current : list, returned : list):
    """Merge two lists of expression values."""
    
    result = []
    if returned:
        if not current:
            return returned
        
        for current_val in current:
            
            for returned_val in returned:
                result.append(f'{current_val}{returned_val}')
    else:
        return current
        
    return result

def expand_bracket(input_values : list, searching : list = None):
    current= []
    result = []
    if searching is None:
        searching = []
    
    while input_values:
        
        #Pop the list as a queue to get the next character to process
        val = input_values.pop(0)
        
        if val == '{':
            if searching and searching[-1] == ',':
                searching.pop(-1)
                
            searching.append('}')
            
            returned = expand_bracket(input_values, searching)
            current = merge_values(current,returned)
            
        elif val == '}':
            if searching and searching[-1] == '}':
                    searching.pop(-1)
            else:
                raise ValueError(f'Invalid input - The character "{val}" was not expected.')
                
            result += current
            
            return result
        
        elif val == ',':
            if '}' not in searching:
                raise ValueError(f'Invalid input - The character "{val}" is not within {{}}')
            
            searching.append(',')
            result += current
            current.clear()
            
        elif val.isalpha():
            
            if searching and searching[-1] == ',':
                searching.pop(-1)
            
            current=merge_values(current,[val])
            
        else:
            raise ValueError(f'Invalid input - The character "{val}" does not match [a-zA-Z,{{}}].')
    
        last_char = val
    
    if searching:
        raise ValueError(f'Invalid input - Expected character(s) {searching}.')
    
    result += current
    
    if not result:
        raise ValueError(f'Invalid input - Expressions cannot be empty.')
    
    return result

def apply_expand_bracket(input_string):
    
    try:
        result = expand_bracket(list(input_string))
        result = ' '.join(result)
    except ValueError as e:
        result = None
        # print(e)
        
    return result    

check_solution(cases, apply_expand_bracket)

Case # 0 | Input: {A,B,C} | A B C == A B C | Runtime: 2.1457672119140625e-05 
Case # 1 | Input: AB{C,D} | ABC ABD == ABC ABD | Runtime: 1.811981201171875e-05 
Case # 2 | Input: {C,D}AB | CAB DAB == CAB DAB | Runtime: 1.5497207641601562e-05 
Case # 3 | Input: {A,B}{C,D} | AC AD BC BD == AC AD BC BD | Runtime: 1.7642974853515625e-05 
Case # 4 | Input: {A,B{C,D}} | A BC BD == A BC BD | Runtime: 1.4066696166992188e-05 
Case # 5 | Input: {{A},{B}} | A B == A B | Runtime: 1.2636184692382812e-05 
Case # 6 | Input: {ABC} | ABC == ABC | Runtime: 9.298324584960938e-06 
Case # 7 | Input: {ABC | None == None | Runtime: 1.6689300537109375e-05 
Case # 8 | Input: {A,B,C}{A,B{{A}}} | AA ABA BA BBA CA CBA == AA ABA BA BBA CA CBA | Runtime: 2.4080276489257812e-05 
Case # 9 | Input: D{A,B}{A,B{{C}}} | DAA DABC DBA DBBC == DAA DABC DBA DBBC | Runtime: 2.0503997802734375e-05 
Case # 10 | Input: {A,B}{{C,D}EF} | ACEF ADEF BCEF BDEF == ACEF ADEF BCEF BDEF | Runtime: 2.6226043701171875e-05 
Case # 11 | Input: